<a href="https://colab.research.google.com/github/UmeshGayashan/OCR_RegNo_Detection/blob/main/OCR_RegNo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
## Go to the drive and get the data
```



In [10]:
# !-shell command, -q -quiet mode, -U -upgrade
# download files and folders from Google Drive
!pip install -qU gdown

# Mount Google Drive directly
from google.colab import drive
drive.mount('/content/gdrive')

# Verify dataset access using native path
import os

dataset_path = '/content/gdrive/MyDrive/OCR_dataset'
if os.path.exists(dataset_path):
    print(f"Found {len(os.listdir(dataset_path))} files:")
    for idx, file in enumerate(os.listdir(dataset_path), 1):
        print(f"{idx}. {file}")
else:
    print("Error: Folder not found. Check sharing permissions and folder name.")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found 7 files:
1. AB_1242_4321.jpg
2. EG_2020_4433.jpg
3. EG_4321_7885.jpg
4. EG_3211_5687.jpg
5. EG_5242_9321.jpg
6. EG_5678_1234.jpg
7. CD_5496_8753.jpg
